In [0]:
!pip install tensorflow-gpu

In [6]:
# save the final model to file
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten,Dropout
from keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.3))
  model.add(Dense(10, activation='softmax'))
	# compile model
  opt = SGD(lr=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=1)
	# save model
	model.save('final_model.h5')

# entry point, run the test harness
run_test_harness()

W0622 05:26:01.524224 140258938906496 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch 1/10
60000/60000 [==============================] - 8s 132us/step - loss: 0.2521 - acc: 0.9238
Epoch 2/10
60000/60000 [==============================] - 7s 124us/step - loss: 0.1107 - acc: 0.9659
Epoch 3/10
60000/60000 [==============================] - 7s 124us/step - loss: 0.0833 - acc: 0.9745
Epoch 4/10
60000/60000 [==============================] - 8s 127us/step - loss: 0.0681 - acc: 0.9792
Epoch 5/10
60000/60000 [==============================] - 8s 125us/step - loss: 0.0581 - acc: 0.9821
Epoch 6/10
60000/60000 [==============================] - 7s 123us/step - loss: 0.0499 - acc: 0.9838
Epoch 7/10
60000/60000 [==============================] - 7s 123us/step - loss: 0.0468 - acc: 0.9853
Epoch 8/10
60000/60000 [==============================] - 7s 122us/step - loss: 0.0429 - acc: 0.9860
Epoch 9/10
60000/60000 [==============================] - 7s 122us/step - loss: 0.0397 - acc: 0.9867
Epoch 10/10
60000/60000 [==============================] - 7s 122us/step - loss: 0.0361 - a

In [7]:
import pandas as pd
from keras.models import load_model

df = pd.read_csv("test.csv")
dataset = df.values

model = load_model('final_model.h5')
result = []
for i in range(28000):
  pixels = dataset[i].reshape(1,28,28,1)
  pixels = pixels.astype("float32")
  pixels = pixels/255.0
  r = model.predict_classes(pixels)
  result.append(r[0])
print(len(result))

28000


In [0]:
import numpy as np

vals = np.round(result)
range = np.arange(1,28001)
with open("output.csv", "w") as f:
    f.write("ImageId,Label\n")
    for x, y in zip(range, vals):
        f.write("{},{}\n".format(x, y))
f.close()